# Local Ollama model on MedSynth data

In [1]:
import helpers
import ollama
import pandas as pd
from tqdm import tqdm


In [2]:
df_ms = helpers.process_medsynth()
df = helpers.extract_vitalsigns_tocols(df_ms)
df = helpers.remove_doctors_dialogue(df)

df

✔ File already exists: data/MedSynth_huggingface_final.csv


,Note,Dialogue,ICD10,ICD10_desc,ICD_chapter,BP_systolic,BP_diastolic,Heart_Rate,Respiratory_Rate,temp,temp_unit,Oxygen_Saturation,Oxygen_Device,temp_c,temp_f,patient_only
0,**1. Subjective:**\n\n **Chief Complaint (CC...,[doctor]: Hello! It’s good to see you today. H...,M25562,PAIN IN LEFT KNEE,Diseases of musculoskeletal system,128.0,82.0,72.0,16.0,98.6,f,NaN,NaN,37.000000,98.6,": Hi, Doctor. I’ve been having a lot of pain i..."
1,**1. Subjective:**\n\n - **Chief Complaint (...,"[doctor] Hi there, how are you today?\n\n[pati...",M25562,PAIN IN LEFT KNEE,Diseases of musculoskeletal system,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hi doctor, I'm doing alright, but I've been h..."
2,**1. Subjective:**\n\n**Chief Complaint (CC):*...,"[doctor] Good morning, how are you doing today...",M25562,PAIN IN LEFT KNEE,Diseases of musculoskeletal system,140.0,85.0,NaN,16.0,98.6,f,NaN,NaN,37.000000,98.6,"Good morning, Doctor. I've been better, to be..."
3,**1. Subjective:**\n\n**Chief Complaint (CC):*...,[doctor] Good morning! How are you feeling tod...,M25562,PAIN IN LEFT KNEE,Diseases of musculoskeletal system,130.0,80.0,72.0,16.0,98.6,f,NaN,NaN,37.000000,98.6,"Good morning, doctor. I've been better. I'm h..."
4,#####\n**1. Subjective:**\n\n**Chief Complaint...,"[doctor]: Hello Mr. Doe, how are you doing tod...",M25562,PAIN IN LEFT KNEE,Diseases of musculoskeletal system,130.0,80.0,72.0,16.0,98.6,f,NaN,NaN,37.000000,98.6,": Hello doctor. I’m alright, although my knee ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,#####\n**1. Subjective:**\n \n**Chief Compla...,[doctor]: Good morning. How are you doing toda...,B3781,CANDIDAL ESOPHAGITIS,Certain infectious and parasitic diseases,120.0,80.0,82.0,18.0,98.6,f,NaN,NaN,37.000000,98.6,": Not so good, doctor. I've been having a real..."
10236,### Gastroenterologist Medical Note\n\n#### 1....,"**Doctor:** Hi there, how are you doing today?...",B3781,CANDIDAL ESOPHAGITIS,Certain infectious and parasitic diseases,130.0,85.0,72.0,NaN,98.6,f,NaN,NaN,37.000000,98.6,
10237,**1. Subjective:**\n\n**Chief Complaint (CC):*...,"[doctor]: Hi Mr. Harris, how are you doing tod...",B3781,CANDIDAL ESOPHAGITIS,Certain infectious and parasitic diseases,122.0,78.0,88.0,18.0,98.6,f,98.0,room air,37.000000,98.6,": Hi, Doctor. I'm not feeling great, to be hon..."
10238,#####\n**1. Subjective:**\n**Chief Complaint (...,"[doctor]: Good morning, Ms. Lee. How are you d...",B3781,CANDIDAL ESOPHAGITIS,Certain infectious and parasitic diseases,120.0,80.0,72.0,NaN,98.6,f,NaN,NaN,37.000000,98.6,": Good morning, doctor. I'm not feeling well a..."


In [40]:
MODEL = "tinyllama"

def ollama_predict(prompt, model=MODEL):
    response = ollama.generate(model=model, prompt=prompt)
    return response["response"]


allowed = df["ICD_chapter"].unique().tolist()

def restrict_output(output):
    for a in allowed:
        if output.lower() == a.lower():
            return a
    return "UNKNOWN"


def build_prompt(row):
    return f"""
You are a medical classification model.

Classify the ICD chapter of this case.

Dialogue:
{row['patient_only']}

Vitals:
BP: {row['BP_systolic']}/{row['BP_diastolic']}
HR: {row['Heart_Rate']}
RR: {row['Respiratory_Rate']}
Temp C: {row['temp_c']}
SpO2: {row['Oxygen_Saturation']}
Oxygen Device: {row['Oxygen_Device']}

Answer ONLY with the ICD chapter name from this list {allowed}.
"""




In [41]:
df_test = df[0:20]

In [ ]:

preds = []

for idx, row in tqdm(df_test.iterrows(), total=len(df_test)):
    prompt = build_prompt(row)
    llm_output = ollama_predict(prompt)

    preds.append(llm_output.strip())



true_labels = df["ICD_chapter"].astype(str).str.strip()
pred_labels = pd.Series(preds)


  0%|          | 0/20 [00:00<?, ?it/s]

<class 'str'>

You are a medical classification model.

Classify the ICD chapter of this case.

Dialogue:
[doctor]: Hello! It’s good to see you today. How can I help you?

[patient]: Hi, Doctor. I’ve been having a lot of pain in my left knee.

[doctor]: I’m sorry to hear that. Can you tell me more about the pain and how long it’s been going on?

[patient]: Sure. It's been pretty bad, moderate to severe pain, for the last three weeks.

[doctor]: Hmm, that sounds uncomfortable. Is there anything specific that makes the pain worse?

[patient]: Yes, physical activities really make it worse. Walking, climbing stairs, standing for long periods—it all makes it more painful.

[doctor]: I see. Have you noticed any swelling or stiffness in the knee?

[patient]: Yes, there’s occasional swelling and it’s really stiff, especially in the mornings.

[doctor]: Alright, any other symptoms you’ve noticed? Fever, weight loss, anything like that?

[patient]: No, no fever or weight loss.

[doctor]: How abo